# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import pandas as pd
import sqlite3

conn = sqlite3.connect('data.sqlite')

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [2]:
# Your code here
q = """

SELECT e.lastName, e.firstname, o.city
  FROM employees AS e
  JOIN offices AS O
 USING (officeCode)
 WHERE o.city = 'Boston';


"""

pd.read_sql(q, conn)

,lastName,firstName,city
0,Firrelli,Julie,Boston
1,Patterson,Steve,Boston


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [3]:
# Your code here
q = """

   SELECT o.officeCode, o.city, COUNT(e.employeeNumber) AS numOfEmployees
     FROM offices AS o
LEFT JOIN employees AS e
    USING (officeCode)
 GROUP BY officeCode


"""

pd.read_sql(q, conn)

,officeCode,city,numOfEmployees
0,1,San Francisco,6
1,2,Boston,2
2,3,NYC,2
3,4,Paris,5
4,5,Tokyo,2
5,6,Sydney,4
6,7,London,2
7,27,Boston,0


## Write 3 questions of your own and answer them

In [4]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""

'\nHow many customers are there per office?\n'

In [5]:
"""
How many employees are there per country?
"""

# Your code here

q = """

   SELECT o.country, COUNT(e.employeeNumber) AS numOfEmployees
     FROM offices AS o
LEFT JOIN employees AS e
    USING (officeCode)
 GROUP BY o.country


"""

pd.read_sql(q, conn)

,country,numOfEmployees
0,Australia,4
1,France,5
2,Japan,2
3,UK,2
4,USA,10


In [6]:
"""
How many customers are there per sales rep?
"""

# Your code here

q = """

   SELECT e.firstName || ' ' || e.lastName AS employeeName, COUNT(c.customerNumber) AS numOfCustomers
     FROM employees AS e
LEFT JOIN customers AS c
       ON c.salesRepEmployeeNumber = e.employeeNumber
GROUP BY e.employeeNumber


"""

pd.read_sql(q, conn)

,employeeName,numOfCustomers
0,Diane Murphy,0
1,Mary Patterson,0
2,Jeff Firrelli,0
3,William Patterson,0
4,Gerard Bondur,0
5,Anthony Bow,0
6,Leslie Jennings,6
7,Leslie Thompson,6
8,Julie Firrelli,6
9,Steve Patterson,6


In [7]:
"""
How many orders are there per customer?
"""

# Your code here

q = """

    SELECT c.customerName, COUNT(o.orderNumber)
      FROM customers AS c
 LEFT JOIN orders AS o
     USING (customerNumber)
  GROUP BY c.customerNumber


"""

pd.read_sql(q, conn)

,customerName,COUNT(o.orderNumber)
0,Atelier graphique,3
1,Signal Gift Stores,3
2,"Australian Collectors, Co.",5
3,La Rochelle Gifts,4
4,Baane Mini Imports,4
...,...,...
117,Motor Mint Distributors Inc.,3
118,Signal Collectibles Ltd.,2
119,"Double Decker Gift Stores, Ltd",2
120,Diecast Collectables,2


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [9]:
# Your code here

q = """

  SELECT e.firstName || ' ' || e.lastName, p.productName
    FROM employees AS e
         JOIN customers AS c
           ON e.employeeNumber = c.salesRepEmployeeNumber
         JOIN orders AS o
        USING (customerNumber)
         JOIN orderdetails AS od
        USING (orderNumber)
         JOIN products AS p
        USING (productCode)



"""

pd.read_sql(q, conn)

,e.firstName || ' ' || e.lastName,productName
0,Leslie Jennings,1958 Setra Bus
1,Leslie Jennings,1940 Ford Pickup Truck
2,Leslie Jennings,1939 Cadillac Limousine
3,Leslie Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie Jennings,1968 Ford Mustang
...,...,...
2991,Martin Gerard,1954 Greyhound Scenicruiser
2992,Martin Gerard,1950's Chicago Surface Lines Streetcar
2993,Martin Gerard,Diamond T620 Semi-Skirted Tanker
2994,Martin Gerard,1911 Ford Town Car


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [14]:
# Your code here

q = """

SELECT e.firstName || ' ' || e.lastName AS employeeName, SUM(od.quantityOrdered) AS numOfProductsSold
  FROM employees AS e
       JOIN customers AS c
         ON e.employeeNumber = c.salesRepEmployeeNumber
       JOIN orders AS o
      USING (customerNumber)
       JOIN orderdetails AS od
      USING (orderNumber)
GROUP BY e.employeeNumber
ORDER BY e.lastName, e.firstName


"""

pd.read_sql(q, conn)

,employeeName,numOfProductsSold
0,Loui Bondur,6186
1,Larry Bott,8205
2,Pamela Castillo,9290
3,Julie Firrelli,4227
4,Andy Fixter,6246
5,Martin Gerard,4180
6,Gerard Hernandez,14231
7,Leslie Jennings,11854
8,Barry Jones,7486
9,Peter Marsh,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [26]:
# Your code here

q = """

 SELECT e.firstName || ' ' || e.lastName AS employeeName, COUNT(od.productCode) AS numOfDiffProducts
   FROM employees AS e
        JOIN customers AS c
          ON e.employeeNumber = c.salesRepEmployeeNumber
        JOIN orders AS o
       USING (customerNumber)
        JOIN orderdetails AS od
       USING (orderNumber)
GROUP BY e.employeeNumber
  HAVING COUNT(od.productCode) > 200
ORDER BY e.lastName, e.firstName;


"""

pd.read_sql(q, conn)

,employeeName,numOfDiffProducts
0,Larry Bott,236
1,Pamela Castillo,272
2,Gerard Hernandez,396
3,Leslie Jennings,331
4,Barry Jones,220
5,George Vanauf,211


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!